Week 8 exploration

# Setup

In [ ]:
import os
import glob
import itertools as it

import cv2
%matplotlib inline 
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

from PIL import Image

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from pylibdmtx.pylibdmtx import decode
from ultralytics import YOLO, settings
root_dir = os.getcwd().replace('\\notebooks', '')
settings.update({'datasets_dir': f'{root_dir}/data/processed/test', 'runs_dir': f'{root_dir}/yolo/runs'})

# 1 - Histogram Analysis

From YOLO crops, create histogram plots and analyse all test images.

In [ ]:
# Getting cropped images
model = YOLO('../yolo/models/ultralytics_finetuned.pt')

# All MAN images
img_paths = glob.glob('../data/extracted/*')
img_paths.remove('../data/extracted\\info.csv')

images = []
for img_path in img_paths:
    img = cv2.imread(img_path)
    results = model(img)
    boxes = results[0].boxes.xyxy.tolist()

    if boxes != None and len(boxes) > 0:
        # only look at box with highest confidence
        box = boxes[0]

        # crop with some padding (to not have too small of a crop)
        pad = 45
        crop_obj = img[max(0, int(box[1])-pad):max(0, int(box[3])+pad), max(0, int(box[0])-pad):max(0, int(box[2])+pad)]

        images.append(crop_obj)

In [ ]:
len(images)

In [ ]:
# create histograms from cropped images
def create_hist(img):
    # grayscale image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    return hist

hists = [create_hist(img) for img in images]

# combine histograms with images
for idx, img_hist in enumerate(zip(images, hists)):
    img, hist = img_hist
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Subplot of image and histogram side by side with equal dimensions and their axis removed
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.axis('off') # remove axis
    plt.imshow(img)
    plt.subplot(1, 2, 2)
    plt.plot(hist)
    plt.xlabel('Grayscale Pixel Intensity')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

In [ ]:
# Binarize with Otsu's method
def binarize(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return img

binarized_imgs = [binarize(img) for img in images]

# combine binarized images with original images
for idx, img_bin in enumerate(zip(images, binarized_imgs)):
    img, img_bin = img_bin
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.subplot(1, 2, 2)
    plt.imshow(img_bin, cmap='gray')
    plt.tight_layout()
    plt.show()
    break

## Pipeline with step 2 Otsu's binarization

In [ ]:
# Getting cropped images
model = YOLO('../yolo/models/ultralytics_finetuned.pt')

# MAN test data
img_paths = glob.glob('../data/MAN/images/test/*.jpg')

# stat tracking
num_decodings = 0
num_valid_decodings = 0

imgs = []
titles = []

idx = 0
for img_path in img_paths:
    img = cv2.imread(img_path)
    results = model(img)
    boxes = results[0].boxes.xyxy.tolist()

    actual_decoding = os.path.basename(img_path).split('_')[0]
    actual_decoding = actual_decoding.replace('-', '') # minor cleaning

    # fixing some actual decodings
    if actual_decoding == 'KV8INMEP':
        actual_decoding = '#D1FPE50HA9NS0047XG264##KV8INMEP'
    elif actual_decoding == 'KW8PXY3D':
        actual_decoding = '#D1FPE50HA9NS0047XG264##KW8PXY3D'

    if boxes != None and len(boxes) > 0:
        # only look at box with highest confidence
        box = boxes[0]

        # crop with some padding (to not have too small of a crop)
        pad = 45
        crop_obj_pre_binarize = img[max(0, int(box[1])-pad):max(0, int(box[3])+pad), max(0, int(box[0])-pad):max(0, int(box[2])+pad)]

        # binarize with otsu method
        crop_obj = binarize(crop_obj_pre_binarize)

        # show pre and post binarization crops
        plt.subplot(1, 2, 1)
        plt.imshow(crop_obj_pre_binarize)
        plt.subplot(1, 2, 2)
        plt.imshow(crop_obj, cmap='gray')
        plt.show()

        decodings = decode(crop_obj)

        if decodings != None and len(decodings) > 0:
            num_decodings += 1

            for decoding in decodings:
                decoded_string = decoding.data.decode('utf-8')
                
                if decoded_string == actual_decoding:
                    print(f'Valid decoding! {actual_decoding}')
                    num_valid_decodings += 1
                    imgs.append(crop_obj)
                    titles.append(actual_decoding)
                else:
                    print('Invalid decoding!')
                    print(actual_decoding)
                    print(decoded_string)

        # optional saving
        # cv2.imwrite(f'../data/cropped/{actual_decoding}-{idx}.jpg', crop_obj)
        idx += 1

# calculate stats
dm_decode_rate = num_decodings/len(img_paths)
valid_decode_rate = num_valid_decodings/len(img_paths)

print()
print(f'{num_valid_decodings}/{len(img_paths)}')
print(f'Dm decode rate: {dm_decode_rate}')
print(f'Valid decode rate: {valid_decode_rate}')